In [26]:
import glob
import numpy as np
import xarray as xr
import pandas as pd
routelink_subset_file = "/home/APD/inland_hydraulics/wrf-hydro-run/DOMAIN/routeLink_subset.nc" 

usgs_timeslices_folder = "/home/jacob.hreha/github/troute/t-route/test/input/geo/nudgingTimeSliceObs/"

usgs_file_pattern_filter = "2020-03-21*.usgsTimeSlice.ncdf"

usgs_files = glob.glob(usgs_timeslices_folder + usgs_file_pattern_filter)


In [41]:
%%timeit
def read_netcdfs(files, dim, transform_func=None):
    def process_one_path(path):
        # use a context manager, to ensure the file gets closed after use
        with xr.open_dataset(path) as ds:
            # transform_func should do some sort of selection or
            # aggregation
            if transform_func is not None:
                ds = transform_func(ds)
            # load all data from the transformed dataset, to ensure we can
            # use it after closing each original file
            ds.load()
            return ds

    paths = sorted(glob.glob(files))
    datasets = [process_one_path(p) for p in paths]
    combined = xr.concat(datasets, dim)
    return combined

# here we suppose we only care about the combined mean of each file;
# you might also use indexing operations like .sel to subset datasets
# combined = read_netcdfs('/all/my/files/*.nc', dim='time',
#                         transform_func=lambda ds: ds.mean())

with read_netcdfs(usgs_timeslices_folder + usgs_file_pattern_filter,'time',add_time_dimension) as ds3:
    df3 = pd.DataFrame(ds3['discharge'].values,index=ds3['stationId'].values,columns=ds3.time.values)


10.3 s ± 283 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# df3

In [44]:
%%timeit
def add_time_dimension(ds):
    t = [ds['queryTime'].values[0]]
    time_da = xr.DataArray(t,[('time',t)])
    return ds.drop_vars('time').expand_dims(time=time_da,axis=1)

with xr.open_mfdataset(usgs_files,preprocess=add_time_dimension, combine= "nested", concat_dim='time') as ds2:
    df2 = pd.DataFrame(ds2['discharge'].values,index=ds2['stationId'].values,columns=ds2.time.values)

df2 



9.47 s ± 138 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [66]:
ds = xr.open_dataset(usgs_files[0])
# ds = ds.reset_index('stationIdInd',drop=True)
t = [ds['queryTime'].values[0]]
time_da = xr.DataArray(t,[('time',t)])
stationId_da = ds.stationId
ds2 = ds.drop_vars('time').expand_dims(time=time_da,axis=1)
ds2 = ds2.drop_vars('stationId').expand_dims(stationId=stationId_da,axis=1)
ds,ds2

(<xarray.Dataset>
 Dimensions:            (stationIdInd: 3869)
 Dimensions without coordinates: stationIdInd
 Data variables:
     stationId          (stationIdInd) |S15 b'       08158930' ... b'       05...
     time               (stationIdInd) |S19 ...
     discharge          (stationIdInd) float32 -999999.0 -999999.0 ... -999999.0
     discharge_quality  (stationIdInd) int16 100 100 100 100 ... 100 100 100 100
     queryTime          (stationIdInd) datetime64[ns] 2020-03-21T13:30:00 ... ...
 Attributes:
     sliceTimeResolutionMinutes:  15
     fileUpdateTimeUTC:           2020-03-21_13:30:00
     sliceCenterTimeUTC:          2020-03-21_13:30:00, <xarray.Dataset>
 Dimensions:            (stationId: 3869, stationIdInd: 3869, time: 1)
 Coordinates:
   * stationId          (stationId) |S15 b'       08158930' ... b'       05374...
   * time               (time) datetime64[ns] 2020-03-21T13:30:00
 Dimensions without coordinates: stationIdInd
 Data variables:
     discharge          (sta

In [ ]:

# def get_usgs_from_wrf_hydro(routelink_subset_file,usgs_timeslices_folder):

with xr.open_dataset(routelink_subset_file) as ds:
    df = ds.to_dataframe()
    df = df.reset_index()
    df.drop(df.columns.difference(['link','to','gages','ascendingIndex']), 1, inplace=True)
    df['gages'] = df['gages'].str.decode('utf-8')
    nan_value = float("NaN")
    df['gages'].replace('               ', nan_value, inplace=True)
    df = df[df['gages'].notna()]
    df.dropna(subset=['gages'], inplace=True)
    df = df.set_index('gages')


In [43]:
%%timeit
with xr.open_mfdataset(usgs_files,combine='nested',concat_dim='stationIdInd') as ds:
    df2 = ds.to_dataframe()
    df2['stationId'] = df2['stationId'].str.decode('utf-8') 
    df2['time'] = df2['time'].str.decode('utf-8') 
    df2 = df2.reset_index()
    df2.drop(df2.columns.difference(['stationId','time','discharge','discharge_quality']), 1, inplace=True)
    df2.reset_index()
    df2.set_index('stationId', inplace=True)



14.4 s ± 606 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:

usgs_df = df2.join(df)
usgs_df = usgs_df.reset_index()
usgs_df = usgs_df[usgs_df['index'].notna()]
nan_value = float("NaN")
usgs_df['index'].replace(['               ','            N/A','           NONE'], nan_value, inplace=True)
usgs_df = usgs_df[usgs_df['index'].notna()]
# pd.to_numeric(usgs_df['index']) 
usgs_df = usgs_df.set_index('index')
usgs_df = usgs_df.reset_index()
usgs_df = usgs_df.set_index('link')

usgs_df = usgs_df.pivot_table(values='discharge', index=usgs_df.index, columns='time', aggfunc='first')
usgs_df.index = usgs_df.index.astype('int')
columns_list = (usgs_df.columns)



In [17]:

for i in range(0,(len(columns_list)*3)-12,12):
    original_string = usgs_df.columns[i]
    original_string_shortened = original_string[:-5]
    temp_name1 = original_string_shortened + str('05:00')
    temp_name2 = original_string_shortened + str('10:00')
    temp_name3 = original_string_shortened + str('20:00')
    temp_name4 = original_string_shortened + str('25:00')
    temp_name5 = original_string_shortened + str('35:00')
    temp_name6 = original_string_shortened + str('40:00')
    temp_name7 = original_string_shortened + str('50:00')
    temp_name8 = original_string_shortened + str('55:00')
    usgs_df.insert(i+1, temp_name1, np.nan)
    usgs_df.insert(i+2, temp_name2, np.nan)
    usgs_df.insert(i+4, temp_name3, np.nan)
    usgs_df.insert(i+5, temp_name4, np.nan)
    usgs_df.insert(i+7, temp_name5, np.nan)
    usgs_df.insert(i+8, temp_name6, np.nan)
    usgs_df.insert(i+10, temp_name7, np.nan)
    usgs_df.insert(i+11, temp_name8, np.nan)

    # usgs_df = usgs_df.astype(float)



usgs_df = usgs_df.interpolate(method='linear',  axis=1)
usgs_df
# get_usgs_from_wrf_hydro(routelink_subset_file,usgs_timeslices_folder)

time,2020-03-19_18:00:00,2020-03-19_18:05:00,2020-03-19_18:10:00,2020-03-19_18:15:00,2020-03-19_18:20:00,2020-03-19_18:25:00,2020-03-19_18:30:00,2020-03-19_18:35:00,2020-03-19_18:40:00,2020-03-19_18:45:00,...,2020-03-24_17:15:00,2020-03-24_17:20:00,2020-03-24_17:25:00,2020-03-24_17:30:00,2020-03-24_17:35:00,2020-03-24_17:40:00,2020-03-24_17:45:00,2020-03-24_17:50:00,2020-03-24_17:55:00,2020-03-24_18:00:00
link,,,,,,,,,,,,,,,,,,,,,
3923,74.756477,74.756477,74.756477,74.756477,74.756477,74.756477,74.756477,74.756477,74.756477,74.756477,...,124.877296,124.877296,124.877296,124.877296,124.877296,124.877296,124.877296,124.877296,124.877296,124.877296
12932,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,...,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000
13544,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,...,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000
188173,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,...,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000
188209,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,...,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131001153,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,...,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000
1131001751,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,...,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000
1131002328,255.701126,255.543808,255.386490,255.229172,255.071859,254.914546,254.757233,254.599915,254.442596,254.285278,...,148.663452,148.663452,148.663452,148.663452,148.663452,148.663452,148.663452,148.663452,148.663452,148.663452
